In [34]:
import pandas as pd
import numpy as np
import math
from copy import copy
import os
from numpy.linalg import inv
from numpy import matmul as mx

In [35]:
# ============================================================
#   Author  :   Jhon Charaja
#   Info    :   Kalman 1st and 2nd derivator
# ============================================================


class MultipleKalmanDerivator:
    def __init__(self, deltaT, ndof = 7):
        self.x_v = np.zeros(ndof)
        self.dx_v = np.zeros(ndof)
        self.ddx_v = np.zeros(ndof)
        self.deltaT = deltaT

        self.derivators =   [KalmanDerivator(self.deltaT),
                             KalmanDerivator(self.deltaT), 
                             KalmanDerivator(self.deltaT), 
                             KalmanDerivator(self.deltaT), 
                             KalmanDerivator(self.deltaT),
                             KalmanDerivator(self.deltaT), 
                             KalmanDerivator(self.deltaT),
                             ]

    def update(self, x_update):
        x_v = []
        dx_v = []
        ddx_v = []

        for i, _ in enumerate(self.derivators):
            x, dx, ddx = self.derivators[i].kalman_filter(x_update[i])
            x_v.append(x)
            dx_v.append(dx)
            ddx_v.append(ddx)

        self.x_v = np.array(x_v)
        self.dx_v = np.array(dx_v)
        self.ddx_v = np.array(ddx_v)

        return self.x_v, self.dx_v, self.ddx_v 


class KalmanDerivator:
    def __init__(self, deltaT):
        self.deltaT = deltaT
        
        self.x_k_k = np.zeros((3,1))
        self.x_k1_k = np.zeros((3,1))
        
        self.P_k_k =  np.eye(3)
        self.P_k1_k = np.eye(3)
        
        self.K = np.zeros((3,1))    #np.random.randn(3,1)
        
        
        self.Q = .1*np.array([[deltaT**5/20, deltaT**4/8, deltaT**3/6],
                            [deltaT**4/8, deltaT**3/3, deltaT**2/2],
                            [deltaT**3/6, deltaT**2/2, deltaT]]) #

                #2*np.diag([0.01,0.05,0.1])#
                #2.*np.eye(3)
                
        
        self.R = 0.001*np.eye(1)
        self.I = np.eye(3)
        

    def kalman_filter(self, pos):
        F = np.array([[1., self.deltaT, 0.5*self.deltaT**2],[0., 1., self.deltaT],[0.,0.,1.]])
        H = np.array([[1., 0., 0.]])

        z = np.array([[pos]])

        #Prediction
        self.x_k1_k = mx(F,self.x_k_k)
        self.P_k1_k = mx(F, mx(self.P_k_k, np.transpose(F))) +  self.Q

        #Update
        self.x_k_k = self.x_k1_k + mx(self.K, (z - mx(H, self.x_k1_k)))
        self.P_k_k = mx(mx((self.I - mx(self.K, H)), self.P_k1_k), np.transpose(self.I - mx(self.K, H))) + mx(self.K, mx(self.R, np.transpose(self.K))) 
        #self.P_k_k = mx(self.I - mx(self.K,H), self.P_k1_k)

        self.K = mx(mx(self.P_k1_k, np.transpose(H)), inv(mx(H, mx(self.P_k1_k, np.transpose(H))) + self.R))   
        
        return self.x_k_k[0][0], self.x_k_k[1][0], self.x_k_k[2][0]




In [18]:
class MultipleKalmanIntegrator:
    def __init__(self, deltaT, ndof = 6):
        self.x_v = np.zeros(ndof)
        self.dx_v = np.zeros(ndof)
        self.ddx_v = np.zeros(ndof)
        self.deltaT = deltaT

        self.integrators =   [KalmanIntegrator(self.deltaT),
                             KalmanIntegrator(self.deltaT), 
                             KalmanIntegrator(self.deltaT), 
                             KalmanIntegrator(self.deltaT), 
                             KalmanIntegrator(self.deltaT),
                             KalmanIntegrator(self.deltaT), 
                             ]

    def update(self, ddx_update):
        x_v = []
        dx_v = []
        ddx_v = []

        for i, _ in enumerate(self.integrators):
            x, dx, ddx = self.integrators[i].kalman_filter(ddx_update[i])
            x_v.append(x)
            dx_v.append(dx)
            ddx_v.append(ddx)

        self.x_v = np.array(x_v)
        self.dx_v = np.array(dx_v)
        self.ddx_v = np.array(ddx_v)

        return self.x_v, self.dx_v, self.ddx_v 


class KalmanIntegrator:
    def __init__(self, deltaT):
        self.deltaT = deltaT
        
        self.x_k_k = np.zeros((3,1))
        self.x_k1_k = np.zeros((3,1))
        
        self.P_k_k =  np.eye(3)
        self.P_k1_k = np.eye(3)
        
        self.K = np.zeros((3,1))    #np.random.randn(3,1)
        
        
        self.Q = .1*np.array([[deltaT**5/20, deltaT**4/8, deltaT**3/6],
                            [deltaT**4/8, deltaT**3/3, deltaT**2/2],
                            [deltaT**3/6, deltaT**2/2, deltaT]]) #

                #2*np.diag([0.01,0.05,0.1])#
                #2.*np.eye(3)
                
        
        self.R = 0.001*np.eye(1)
        self.I = np.eye(3)
        

    def kalman_filter(self, pos):
        F = np.array([[1., self.deltaT, 0.5*self.deltaT**2],[0., 1., self.deltaT],[0.,0.,1.]])
        H = np.array([[0., 0., 1.]])

        z = np.array([[pos]])

        #Prediction
        self.x_k1_k = mx(F,self.x_k_k)
        self.P_k1_k = mx(F, mx(self.P_k_k, np.transpose(F))) +  self.Q

        #Update
        self.x_k_k = self.x_k1_k + mx(self.K, (z - mx(H, self.x_k1_k)))
        self.P_k_k = mx(mx((self.I - mx(self.K, H)), self.P_k1_k), np.transpose(self.I - mx(self.K, H))) + mx(self.K, mx(self.R, np.transpose(self.K))) 
        #self.P_k_k = mx(self.I - mx(self.K,H), self.P_k1_k)

        self.K = mx(mx(self.P_k1_k, np.transpose(H)), inv(mx(H, mx(self.P_k1_k, np.transpose(H))) + self.R))   
        
        return self.x_k_k[0][0], self.x_k_k[1][0], self.x_k_k[2][0]

In [19]:
path = '../src/prueba_kalman.csv'
df = pd.read_csv(path)

In [36]:
df

,dddx,dddy,dddz,dddw,dddex,dddey,dddez,dddx_d,dddy_d,dddz_d,...,dq3_k,dq4_k,dq5_k,dq6_k,ddq1_k,ddq2_k,ddq3_k,ddq4_k,ddq5_k,ddq6_k
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.004961,0.0,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,182.231990,-153.070539,-787.596569,0.249525,-2.750944,-1.818928,-0.153253,0.000031,-0.004961,0.0,...,-10.852661,2.324775,1.080762,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70.299723,283.820525,1044.792986,-0.395963,116.694144,138.310627,-3.254525,0.000062,-0.004961,0.0,...,-7.414446,5.751734,0.697515,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-75.745918,-16.076704,-55.718088,-0.012037,174.577220,-212.968331,-6.151069,0.000094,-0.004960,0.0,...,-15.578007,-69.854782,-37.507217,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-3128.317505,-1835.517328,-2624.592668,3.486390,-6612.554817,3835.685126,2790.270174,0.000125,-0.004959,0.0,...,-133.422316,-95.049563,-20.482364,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000017,-0.004907,0.000329,0.000021,-0.004800,0.098334,-0.012429,-0.000156,-0.004959,0.0,...,0.078733,-0.008558,-0.008459,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,0.000035,-0.004913,0.000312,0.000020,-0.004688,0.095180,-0.012661,-0.000125,-0.004959,0.0,...,0.076174,-0.008275,-0.008346,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,0.000054,-0.004918,0.000295,0.000020,-0.004578,0.092123,-0.012863,-0.000094,-0.004960,0.0,...,0.073697,-0.008001,-0.008227,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,0.000074,-0.004922,0.000280,0.000019,-0.004470,0.089162,-0.013037,-0.000062,-0.004961,0.0,...,0.071299,-0.007736,-0.008103,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df['dddy'].values

array([ 0.00000000e+00, -1.53070539e+02,  2.83820525e+02, ...,
       -4.91787808e-03, -4.92241681e-03, -4.92650542e-03])

In [32]:
df['dq2']

0        5.641663
1        7.267706
2        5.139992
3      -66.142740
4       70.368266
          ...    
1995    -0.034055
1996    -0.033592
1997    -0.033143
1998    -0.032708
1999    -0.032286
Name: dq2, Length: 2000, dtype: float64